1. CLASS LAYER

In [6]:
from abc import abstractmethod 
 
class Layer: 
    def __init__(self): 
        self.input = None 
        self.output = None 
        self.input_shape = None 
        self.output_shape = None 
        raise NotImplementedError 
 
    @abstractmethod 
    def input(self): 
        return self.input 
 
    @abstractmethod 
    def output(self): 
        return self.output 
 
    @abstractmethod 
    def input_shape(self): 
        return self.input_shape 
 
    @abstractmethod 
    def output_shape(self): 
        return self.output_shape 
 
    @abstractmethod 
    def forward_propagation(self, input): 
        raise NotImplementedError 
 
    @abstractmethod 
    def backward_propagation(self, output_error, learning_rate): 
        raise NotImplementedError 


2. FULL CONNECTED LAYER

In [7]:
# from .layer import Layer 
import numpy as np 
 
class FCLayer(Layer): 
    def __init__(self, input_shape, output_shape): 
        """ 
 
        :param input_shape:  (1, 3) 
        :param output_shape: (1, 4) 
        (1X3) (3x4) => (1, 4) 
        (3, 1) (1, 4) => (3x4) 
        """ 
        self.input_shape = input_shape 
        self.output_shape = output_shape 
        self.weights = np.random.rand(input_shape[1], output_shape[1]) - 0.5 
        self.bias = np.random.rand(1, output_shape[1]) - 0.5 
 
    def forward_propagation(self, input): 
        self.input = input 
        self.output = np.dot(self.input, self.weights) + self.bias 
        return  self.output 
 
    def backward_propagation(self, output_error, learning_rate): 
        curent_layer_err =  np.dot(output_error, self.weights.T) 
        dweight = np.dot(self.input.T, output_error) 
 
        self.weights -= dweight*learning_rate 
        self.bias -= learning_rate*output_error 
 
        return curent_layer_err 

3. CLASS ACTIVATION LAYER

In [8]:
# from .layer import Layer 
 
class ActivationLayer(Layer): 
    def __init__(self, input_shape, output_shape, activation, activation_prime): 
        """ 
        :param input_shape:  đầu vào input mảng (1, 4) 
        :param output_shape:  mảng 
        :param activation:  hàm 
        :param activation_prime:  hàm 
        """ 
        self.input_shape = input_shape 
        self.output_shape = output_shape 
        self.activation = activation 
        self.activation_prime = activation_prime 
 
    def forward_propagation(self, input): 
        self.input = input 
        self.output = self.activation(input) 
        return self.output 
 
    def backward_propagation(self, output_error, learning_rate): 
        return self.activation_prime(self.input)*output_error 

4. CLASS NEURAL NETWORK LAYER

In [9]:
class Network: 
    def __init__(self): 
        self.layers = [] 
        self.loss = None 
        self.loss_prime = None 
 
    def add(self, layer): 
        self.layers.append(layer) 
 
    def setup_loss(self, loss, loss_prime): 
        self.loss = loss 
        self.loss_prime = loss_prime 
 
    def predict(self, input): 
        """ 
 
        :param input: [[1, 3]] = > 1 , [[1, 3], [3, 5], [3, 4]] 
        :return: kết quả dự đoán 
        """ 
        result = [] 
        n = len(input) 
        for i in range(n): 
            output = input[i] 
 
            for layer in self.layers: 
                output = layer.forward_propagation(output) 
            result.append(output) 
 
        return result 
 
    def fit(self, x_train, y_train, learning_rate, epochs): 
 
        n = len(x_train) 
        for i in range(epochs): 
            err = 0 
            for j in range(n): 
                #lan truyen tiến 
                output = x_train[j] 
                for layer in self.layers: 
                    output = layer.forward_propagation(output) 
 
                #tính lỗi của từng 
                err += self.loss(y_train[j], output) 
 
                #lan truyền ngược 
                error = self.loss_prime(y_train[j], output) 
                for layer in reversed(self.layers): 
                    error = layer.backward_propagation(error, learning_rate) 
 
            err = err / n 
 
            print('epoch : %d/%d  err = %f'%(i, epochs, err)) 

5. CẤU HÌNH DỰ ĐOÁN DỰA TRÊN NEURAL NETWORK

In [10]:
#from network.network import Network 
#from layers.FCLayer import FCLayer 
#from layers.activation_layer import ActivationLayer 
import numpy as np 
 
def relu(z): 
    """ 
 
    :param z:  numpy array 
    :return: 0 nếu z <= 0 
         z nếu z > 0 
         [1, -3, 9, -7] => [1, 0, 9, 0] 
    """ 
    return np.maximum(0, z) 
 
def relu_prime(z): 
    """ 
 
    :param z: numpy array 
    :return: array 1, 0 z> 0 => 1, z < 0 => 0 
    """ 
    z[z<0]=0 
    z[z>0]=1 
    return z 
 
def loss(y_true, y_pred): 
    return 0.5*(y_pred-y_true)**2 

def loss_prime(y_true, y_pred): 
    return y_pred-y_true 

x_train = np.array([[[0, 0]], [[0, 1]], [[1, 0]], [[1, 1]]]) 
y_train = np.array([[[0]], [[1]], [[1]], [[0]]]) 
net = Network() 
net.add(FCLayer((1, 2), (1, 3))) 
net.add(ActivationLayer((1, 3), (1, 3), relu, relu_prime)) 
net.add(FCLayer((1, 3), (1, 1))) 
net.add(ActivationLayer((1, 1), (1, 1), relu, relu_prime)) 
net.setup_loss(loss, loss_prime) 
net.fit(x_train, y_train, epochs=1000, learning_rate=0.01) 
out = net.predict([[0, 1]]) 
print(out)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_19528\2237944706.py:52: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('epoch : %d/%d  err = %f'%(i, epochs, err))


epoch : 0/1000  err = 0.250000
epoch : 1/1000  err = 0.250000
epoch : 2/1000  err = 0.250000
epoch : 3/1000  err = 0.250000
epoch : 4/1000  err = 0.250000
epoch : 5/1000  err = 0.250000
epoch : 6/1000  err = 0.250000
epoch : 7/1000  err = 0.250000
epoch : 8/1000  err = 0.250000
epoch : 9/1000  err = 0.250000
epoch : 10/1000  err = 0.250000
epoch : 11/1000  err = 0.250000
epoch : 12/1000  err = 0.250000
epoch : 13/1000  err = 0.250000
epoch : 14/1000  err = 0.250000
epoch : 15/1000  err = 0.250000
epoch : 16/1000  err = 0.250000
epoch : 17/1000  err = 0.250000
epoch : 18/1000  err = 0.250000
epoch : 19/1000  err = 0.250000
epoch : 20/1000  err = 0.250000
epoch : 21/1000  err = 0.250000
epoch : 22/1000  err = 0.250000
epoch : 23/1000  err = 0.250000
epoch : 24/1000  err = 0.250000
epoch : 25/1000  err = 0.250000
epoch : 26/1000  err = 0.250000
epoch : 27/1000  err = 0.250000
epoch : 28/1000  err = 0.250000
epoch : 29/1000  err = 0.250000
epoch : 30/1000  err = 0.250000
epoch : 31/1000  e